In [16]:
#@Yu Yamaoka
#wavファイルにある音声がwavファイル上でいつ再生されたかをcsv保存する。
import wave
import time
import struct
import numpy as np
from pylab import *
import csv

#Define
name = "yamaoka"
voice_time = 100#[ms],移動平均幅、音声のあるおおよその時間サイズ，精度を出すために用いる
dB = 81##被験者のだいたいの声の大きさ
start_n = 38000#処理短縮用
response_time = []

#音声が以降あるなら1を返す
def judge(y,n,samf):#y:音声データ配列,n:配列中の着目点,samf:サンプリング周波数
    v = 2*(0.0001)*math.exp(dB/20)#閾値[v]
    sig = 0 #シグマの略ってことで
    sample = (int)(samf*(voice_time/1000)) #voice_time[ms]声があるなら1を返す
    for i in range(sample):
        sig += abs(y[n+i])

    if(sig >= v*sample): #voice_time後までの振幅の相加平均が閾値vより大きいならture
        return 1 
    else:
        return 0

def check(y,samf):
    flag = 0
    n = start_n
    while(n <= len(y) - samf//2)&(flag==0):
        if(judge(y,n,samf)==1):
            print("start n is "+str(n))
            flag = 1
        if(10*n%samf==0):
            print("now time is :" + str(n/samf)+"[s]")
        n = n + 1
    print("response_time is :"+str(n/samf))
    response_time.append(n/samf)


In [15]:
if __name__ == '__main__':
    j = 0
    for i in range(30): 
        if(i%30==0)&(j<4)&(i>0):
            j  = j + 1
        #wf = wave.open("data/"+name + "/record/" + str(j) + "-" + str(i-j*15+3) + ".wav", "r")
        wf = wave.open("data/"+name + "/record/" + str(1) + "-" + str(i+1) + ".wav", "r")
        fs = wf.getnframes() #総フレーム数
        samf = wf.getframerate() #サンプリング周波数
        print("総フレーム数： " + str(fs))
        print("サンプリング周波数： " + str(samf))
        print("open file is " +name + str(j) + "-" + str(i-j*15+1) + ".wav",)
    
        x = wf.readframes(wf.getnframes())
        x = frombuffer(x, dtype="int16") / 32768.0
    
        check(x,samf)
        print("*************************************")
    
    f = open("data/"+name+"_ResponseTime.csv", 'a')
    writer = csv.writer(f, lineterminator="\n")
    writer.writerow(response_time)
